### 将缩进改为 tab

In [87]:
%%javascript

IPython.tab_as_tab_everywhere = function(use_tabs) {
    if (use_tabs === undefined) {
        use_tabs = true; 
    }

    // apply setting to all current CodeMirror instances
    IPython.notebook.get_cells().map(
        function(c) {  return c.code_mirror.options.indentWithTabs=use_tabs;  }
    );
    // make sure new CodeMirror instances created in the future also use this setting
    CodeMirror.defaults.indentWithTabs=use_tabs;

    };

IPython.tab_as_tab_everywhere()

<IPython.core.display.Javascript object>

In [45]:
#-*- coding:utf-8 -*-

import pandas as pd
import numpy as np

# 预处理

### 提取 品牌为 “美的” 的评论

In [2]:
inputfile = "./data/huizong.csv"

data = pd.read_csv( inputfile , encoding="utf-8" )
print( data.head(3) )

outputfile = "./data/meidi_jd.txt"
data = data[[u"评论"] ][ data[u"品牌"] == u"美的" ]
print( data.head(3) )

data.to_csv( outputfile , index=False , header=False )

   Id    已采     已发 电商平台  品牌                                            评论  \
0   1  True  False   京东  AO                             挺好的 ，安装人员很负责 指的夸奖   
1   2  True  False   京东  AO                              自己安装的，感觉蛮好。后续追加。   
2   3  True  False   京东  AO  \n\nAO史密斯（A.O.Smith） ET300J-60 电热水器 60升 还没安装   

                 时间                                   型号  \
0    2014/8/26 9:07  AO史密斯（A.O.Smith） ET300J-60 电热水器 60升   
1  2014/10/17 14:24  AO史密斯（A.O.Smith） ET300J-60 电热水器 60升   
2  2014/11/12 13:36  AO史密斯（A.O.Smith） ET300J-60 电热水器 60升   

                                             PageUrl  
0  http://s.club.jd.com/productpage/p-1008025-s-0...  
1  http://s.club.jd.com/productpage/p-1008025-s-0...  
2  http://s.club.jd.com/productpage/p-1008025-s-0...  
                                    评论
95898         京东商城信得过，买的放心，用的省心、安心、放心！
95899  给公司宿舍买的，上门安装很快，快递也送的及时，不错的。给五分吧
95900                      美的值得信赖，质量不错


### 文本去重 ：编辑距离、 Simhash 算法

编辑距离：对重复性的，无意义的评论效果很好，但很容易引起误删（尤其是差评，很多评论都很相似），而且距离的阈值也不好确定。
        特别是，两两之间计算编辑，计算量很大
        
SimHash：不同于传统的hash，SimHash是一种LSH(局部敏感哈希)，即语义相似的文本，hash值也相近。可以采用抽屉原理粗过滤，减少候选匹配集进行加速

本文：仅删除完全相同的文本

In [3]:
inputfile = "./data/meidi_jd.txt"
# 去重之后
outputfile = "./data/meidi_jd_process_1.txt"

data = pd.read_csv( inputfile , encoding="utf-8" , header=None  )
print( data.head(3) )
print( "before unique length : " , len( data ))
data = pd.DataFrame( data[0].unique() )
print( "after  unique length : " , len( data ))


data.to_csv( outputfile , index=False , header=False , encoding="utf-8" )

                                 0
0         京东商城信得过，买的放心，用的省心、安心、放心！
1  给公司宿舍买的，上门安装很快，快递也送的及时，不错的。给五分吧
2                      美的值得信赖，质量不错
before unique length :  55400
after  unique length :  53048


### 机械压缩去词

例如“非常好非常好非常好非常好。。”这种词，评论者是为了获取评论的奖励，并没有实际的意义
或者为了强调 “感觉不错，但是实在太贵太贵太贵太贵”。这种情况一般出现在开头或者结尾

所以，需要机械压缩去词

In [79]:
# 去除连续的重复字串
import re
import jieba 
import pandas as pd
def findMinmumSubstring( str ):
    
	str = str.replace( "\n" , "")
	strLen = len( str )

	punc = re.findall(r"[\W]+" , str)
	# print( punc )

	str = re.findall(r"[\w']+", str)
	# print( pd.unique(str) )
	uniStr = pd.unique( str )
    
    

	# 拼接字符串 和 标点符号
	resStr = ""
	for i in uniStr:
		# print (i)
		if ( str.index(i) < len( punc ) ):
# 			print( punc , str.index(i) )
			resStr += i + punc[ str.index( i ) ]
		else:
			resStr += i + " "            
	# print( resStr )
	return resStr

In [83]:
inputfile = "./data/meidi_jd_process_1.txt"
outputfile = "./data/meidi_jd_process_2.txt"

data = pd.read_csv( inputfile , header=None )

data = data.apply(  lambda row: findMinmumSubstring( row[0] ) , axis=1 )
data = data.replace(" ","")
data.to_csv( outputfile , index=False , header=False , encoding="utf-8"  )

data = pd.read_csv( outputfile , header=None , skip_blank_lines=True )
data.to_csv( outputfile , index=False , header=False , encoding="utf-8"  )

### 短句删除

In [86]:
inputfile = "./data/meidi_jd_process_2.txt"
outputfile = "./data/meidi_jd_process_3.txt"

data = pd.read_csv( inputfile , header=None )

print( data.size )

print( data.loc[2000] )
data = np.array( data )

print( data[2000] )
resData = [ i[0] for i in data if len(i[0]) >3 ]

print( pd.DataFrame( resData ).size )
pd.DataFrame( resData ).to_csv( outputfile , index=False , header=False , encoding="utf-8" )


52949
0    还行，就是耗电有点多。
Name: 2000, dtype: object
['还行，就是耗电有点多。']
52856


### 文本评论分词

# 模型构建

### One Hot 
### Distributed Representation